In [1]:
RESULTS_FOLDER = 'results'
COMBINE_ALL = True

In [8]:
!pip install fastcore
!pip install aiohttp
!pip install fastprogress bs4 humanize

     |████████████████████████████████| 97 kB 4.6 MB/s             


In [9]:
from fastprogress import progress_bar
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import re
import humanize
from IPython.display import clear_output, display
from pathlib import Path
import numpy as np
from fastcore.all import *
from time import sleep
import random

import asyncio
import aiohttp

# this is to allow it to run in google colab environment
import nest_asyncio
nest_asyncio.apply()

In [10]:
Path(RESULTS_FOLDER).mkdir(exist_ok=True, parents=True)


In [11]:
def parse_strlist(sl):
    clean = re.sub("[\[\],\s]","",sl)
    splitted = re.split("[\'\"]",clean)
    values_only = [s for s in splitted if s != '']
    return values_only

def get_bitinfochart_graph_values(url, var_name):
  sleep(random.uniform(0, 1.0))
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')

  scripts = soup.find_all('script')
  for script in scripts:
      if 'd = new Dygraph(document.getElementById("container")' in script.text:
          StrList = script.text
          StrList = '[[' + StrList.split('[[')[-1]
          StrList = StrList.split(']]')[0] +']]'
          StrList = StrList.replace("new Date(", '').replace(')','')
          dataList = parse_strlist(StrList)

  date = []
  value = []
  for each in dataList:
      if (dataList.index(each) % 2) == 0:
          date.append(each)
      else:
          value.append(each)

  df = pd.DataFrame(list(zip(date, value)), columns=["date",var_name])
  return df

In [12]:
# This function takes a list of df's then merge's them on the date field with an outer join

def merge_dfs(df_list):
  df_merged = None
  for i in range(len(df_list)-1):
    if i == 0:
      df_merged = df_list[i].merge(df_list[i+1], on='date', how='outer')
    else:
      df_merged = df_merged.merge(df_list[i+1], on='date', how='outer')

  return df_merged

In [13]:
chart_dict_list = [{'url': 'https://bitinfocharts.com/comparison/bitcoin-transactions.html', 'name': 'transactions'},
                    {'url': 'https://bitinfocharts.com/comparison/size-btc.html', 'name': 'block_size'},
                    {'url': 'https://bitinfocharts.com/comparison/sentbyaddress-btc.html', 'name': 'sent_addresses'},
                    {'url': 'https://bitinfocharts.com/comparison/bitcoin-difficulty.html', 'name': 'difficulty'},
                    {'url': 'https://bitinfocharts.com/comparison/bitcoin-hashrate.html', 'name': 'hashrate'},
                    {'url': 'https://bitinfocharts.com/comparison/bitcoin-mining_profitability.html', 'name': 'mining_profitability'},
                   
                    {'url': 'https://bitinfocharts.com/comparison/sentinusd-btc.html', 'name': 'send_usd'},
                    {'url': 'https://bitinfocharts.com/comparison/bitcoin-transactionfees.html', 'name': 'av_transaction_size'},
                    {'url': 'https://bitinfocharts.com/comparison/bitcoin-median_transaction_fee.html', 'name': 'median_transaction_size'},
                    {'url': 'https://bitinfocharts.com/comparison/bitcoin-confirmationtime.html', 'name': 'confirmation_time'},
                    {'url': 'https://bitinfocharts.com/comparison/bitcoin-marketcap.html', 'name': 'market_cap'},
                    {'url': 'https://bitinfocharts.com/comparison/transactionvalue-btc.html', 'name': 'av_transaction_value'},
                   
                    {'url': 'https://bitinfocharts.com/comparison/mediantransactionvalue-btc.html', 'name': 'median_transaction_value'},
                    {'url': 'https://bitinfocharts.com/comparison/tweets-btc.html', 'name': 'tweets'},
                    {'url': 'https://bitinfocharts.com/comparison/google_trends-btc.html', 'name': 'google_trends'},
                    {'url': 'https://bitinfocharts.com/comparison/activeaddresses-btc.html', 'name': 'active_addresses'},
                    {'url': 'https://bitinfocharts.com/comparison/top100cap-btc.html', 'name': 'top_100_percent'},
                    {'url': 'https://bitinfocharts.com/comparison/fee_to_reward-btc.html', 'name': 'fee_reward'},

                    ]

In [14]:
url = 'https://bitinfocharts.com'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

coin_dict_list = []


for span in soup.find_all('span'):
  
  if 's_coins' in str(span.get('class')):
    name = span.get('title').lower()
    coin = span.get('data-coin')
    coin_dict_list.append({'full_name': name,
                'coin': coin})

coin_dict_list[:3]

[{'full_name': 'bitcoin', 'coin': 'btc'},
 {'full_name': 'ethereum', 'coin': 'eth'},
 {'full_name': 'xrp', 'coin': 'xrp'}]

In [15]:
# Combine the list of coins with the list of url's to create a master dictionary with coins and url's to scrape

for coin_dict in coin_dict_list:
  coin_dict['scrape_details'] = []
  for chart_dict in chart_dict_list:
    temp_dict = chart_dict.copy()
    
    url = temp_dict['url']
    url = url.replace('bitcoin', coin_dict['full_name'])
    url = url.replace('btc', coin_dict['coin'])
    url = url.replace(' ', '%20')

    temp_dict['url'] = url
    coin_dict['scrape_details'].append(temp_dict)

coin_dict_list[:2]

[{'full_name': 'bitcoin',
  'coin': 'btc',
  'scrape_details': [{'url': 'https://bitinfocharts.com/comparison/bitcoin-transactions.html',
    'name': 'transactions'},
   {'url': 'https://bitinfocharts.com/comparison/size-btc.html',
    'name': 'block_size'},
   {'url': 'https://bitinfocharts.com/comparison/sentbyaddress-btc.html',
    'name': 'sent_addresses'},
   {'url': 'https://bitinfocharts.com/comparison/bitcoin-difficulty.html',
    'name': 'difficulty'},
   {'url': 'https://bitinfocharts.com/comparison/bitcoin-hashrate.html',
    'name': 'hashrate'},
   {'url': 'https://bitinfocharts.com/comparison/bitcoin-mining_profitability.html',
    'name': 'mining_profitability'},
   {'url': 'https://bitinfocharts.com/comparison/sentinusd-btc.html',
    'name': 'send_usd'},
   {'url': 'https://bitinfocharts.com/comparison/bitcoin-transactionfees.html',
    'name': 'av_transaction_size'},
   {'url': 'https://bitinfocharts.com/comparison/bitcoin-median_transaction_fee.html',
    'name': 'med

In [16]:
coin_merged_df_list = []

for coin_dict in progress_bar(coin_dict_list[:1]):
  print(f"Processing - {coin_dict['full_name']}")
  coin_df_list = []
  for page in progress_bar(coin_dict['scrape_details']):
    try:
      coin_df_list.append(get_bitinfochart_graph_values(url=page['url'], var_name=page['name']))
    except:
      empty_df = pd.DataFrame()
      empty_df['full_name'] = coin_dict['full_name']
      empty_df['coin'] = coin_dict['coin']
      coin_df_list.append(pd.DataFrame)
      print(f"Error with {coin_dict['full_name']}")

  coin_df = merge_dfs(coin_df_list)
  coin_df['full_name'] = coin_dict['full_name']
  coin_df['coin'] = coin_dict['coin']

  coin_merged_df_list.append(coin_df)

  clear_output()

  if not COMBINE_ALL:
    file_path = RESULTS_FOLDER + '/' +coin_dict['full_name'] + '.csv'
    coin_df.to_csv(file_path)
  
if COMBINE_ALL:
  combined_df = pd.concat(coin_merged_df_list, ignore_index=True, sort=False)
  combined_df.to_csv(RESULTS_FOLDER + '/' + 'all_bitcoininfochart_data.csv')

In [17]:
combined_df['full_name'].value_counts()

bitcoin    4846
Name: full_name, dtype: int64

In [24]:
combined_df

,date,transactions,block_size,sent_addresses,difficulty,hashrate,mining_profitability,send_usd,av_transaction_size,median_transaction_size,...,market_cap,av_transaction_value,median_transaction_value,tweets,google_trends,active_addresses,top_100_percent,fee_reward,full_name,coin
0,2009/01/03,null,204,NaN,1,null,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,null,null,NaN,bitcoin,btc
1,2009/01/04,null,null,NaN,null,null,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,null,null,NaN,bitcoin,btc
2,2009/01/05,null,null,NaN,null,null,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,null,null,NaN,bitcoin,btc
3,2009/01/06,null,null,NaN,null,null,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,null,null,NaN,bitcoin,btc
4,2009/01/07,null,null,NaN,null,null,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,null,null,NaN,bitcoin,btc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4841,2022/04/06,273490,622486,454150,28587155782195,2.0298513663513E+20,0.208,49383551581,1.679,0.529,...,848235212918,1028749,576.132,170628,NaN,804316,15.473,1.086,bitcoin,btc
4842,2022/04/07,269266,704504,438482,28587155782195,2.0942859652567E+20,0.181,52894392016,1.62,0.503,...,825844959620,893251,551.355,229396,NaN,784526,15.475,1.151,bitcoin,btc
4843,2022/04/08,266671,675392,400294,28587155782195,1.9441702201934E+20,0.189,55543271543,1.863,0.676,...,823060782668,803142,542.677,224374,NaN,807363,15.47,1.352,bitcoin,btc
4844,2022/04/09,232639,530980,263553,28587155782195,1.9669147591156E+20,0.208,19545003653,1.487,0.528,...,806914076919,608851,470.888,184632,NaN,711262,15.5,0.846,bitcoin,btc


In [ ]:
async def fetch(session, url, full_name, coin, var_name):

  try:
    async with session.get(url) as resp:
        return await resp.text(), full_name, coin, var_name
  except:
    return 'failed', full_name, coin, var_name
        # Catch HTTP errors/exceptions here

In [ ]:
async def fetch_concurrent(coin_dictionary):
  page_results = []
  loop = asyncio.get_event_loop()


  async with aiohttp.ClientSession() as session:
    tasks = []
    for coin_dict in coin_dictionary:
      for page in coin_dict['scrape_details']:

        tasks.append(loop.create_task(fetch(session, 
                                            page['url'], 
                                            coin_dict['full_name'], 
                                            coin_dict['coin'],
                                            page['name'])))

    for result in asyncio.as_completed(tasks):
      text, full_name, coin, var_name = await result
      result_dict = {'response_text': text,
                     'full_name': full_name,
                     'coin': coin,
                     'var_name': var_name}
      page_results.append(result_dict)
            
      
    return page_results

In [ ]:
def extract_info_from_response(response_dict):

  response_text = response_dict['response_text']
  var_name = response_dict['var_name']
  
  soup = BeautifulSoup(response_text, 'html.parser')

  scripts = soup.find_all('script')
  for script in scripts:
      if 'd = new Dygraph(document.getElementById("container")' in script.text:
          StrList = script.text
          StrList = '[[' + StrList.split('[[')[-1]
          StrList = StrList.split(']]')[0] +']]'
          StrList = StrList.replace("new Date(", '').replace(')','')
          dataList = parse_strlist(StrList)

  date = []
  value = []
  for each in dataList:
      if (dataList.index(each) % 2) == 0:
          date.append(each)
      else:
          value.append(each)

  df = pd.DataFrame(list(zip(date, value)), columns=["date",var_name])


  if df[var_name].str.isnumeric().any():
    if df[var_name].str.contains('nullnull').any():
      df[var_name] = np.nan

    return df
  
  else:
    return pd.DataFrame(columns=['date', var_name])

In [ ]:
%%time
http_responses = asyncio.run(fetch_concurrent(coin_dict_list))
len(http_responses)

In [ ]:
coin_merged_df_list = []

for coin_dict in progress_bar(coin_dict_list):
  print(f"Now Processing - {coin_dict['full_name']}")
  coin_df_list = []
  for response in progress_bar(http_responses):
    if coin_dict['full_name'] == response['full_name']:
      coin_df_list.append(extract_info_from_response(response))
  
  coin_df = merge_dfs(coin_df_list)
  coin_df['full_name'] = coin_dict['full_name']
  coin_df['coin'] = coin_dict['coin']
  coin_merged_df_list.append(coin_df)

  file_path = RESULTS_FOLDER + '/' +coin_dict['full_name'] + '__' + coin_dict['coin'] + '.csv'
  coin_df.to_csv(file_path)

  clear_output()

      
combined_df = pd.concat(coin_merged_df_list, ignore_index=True, sort=False)

combined_df.to_csv('all_bitcoininfochart_data.csv')
combined_df.to_pickle('all_bitcoininfochart_data.pkl')

In [ ]:
print('Creating Zip File...')
!zip -r -q results.zip {RESULTS_FOLDER}
print('Complete!')


In [ ]:
df = coin_merged_df_list[52]

In [ ]:
df_columns = df.columns.to_list()

In [ ]:
results_summary_dict_list = []

for df in coin_merged_df_list:
  full_name = df.loc[0,'full_name']
  coin = df.loc[0,'coin']

  total_count = 0
  df_columns = df.columns.to_list()

  for column in df_columns:
    if column != 'date':
      try:
        col_count = df[column].str.isnumeric().sum()
        total_count = col_count + total_count
      except:
        pass
  results = {'full_name': full_name,
             'coin': coin,
             'values': total_count}
  results_summary_dict_list.append(results)

df_summary = pd.DataFrame(results_summary_dict_list)
display(df_summary.sort_values('values', ascending=False).head(20))
df_summary.to_csv('summary.csv')